# Introduction to uDALES Post-processing with Python

This tutorial describes how to read and process data from the LES code uDALES using Python. This tutorial introduces the `UDBase` post-processing class.

## Overview

The **UDBase** post-processing class reads simulation parameters and contains methods to load field and facet data.

**Field data methods:**
- `load_stat_xyt`: Load 1D slab- and time-averaged statistics
- `load_stat_t`: Load 3D time-averaged statistics
- `load_field`: Load instantaneous 3D data
- `load_slice`: Load instantaneous 2D slices

**Facet data methods:**
- `calculate_frontal_properties`: Calculate skylines, frontal areas, blockage ratios
- `plot_fac_type`: Display surface types
- `assign_prop_to_fac`: Assign properties to facets
- `plot_fac`: Display variables on mesh
- `load_fac_momentum`: Load pressure and shear stresses
- `load_fac_eb`: Load energy balance terms
- `load_seb`: Load all SEB terms
- `load_fac_temperature`: Load facet temperatures
- `area_average_seb`, `area_average_fac`: Area averaging
- `time_average`: Time averaging
- `convert_fac_to_field`: Convert facet to 3D field

## 1. Import Libraries and Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Add the uDALES python tools to path
sys.path.insert(0, str(Path.cwd().parent.parent / 'tools' / 'python'))

from udbase import UDBase

print("✓ Libraries imported successfully")

## 2. Initializing UDBase

The starting point is that you have run a simulation and merged the output files. The uDALES postprocessing class is called `UDBase`.

You can initialize it by providing the path to your simulation output directory.

In [ ]:
# Example: Initialize UDBase for experiment 065
expnr = '065'
expdir = Path('../experiments/065')

# Check if directory exists
if expdir.exists():
    sim = UDBase(case_dir=str(expdir))
    print(f"✓ UDBase initialized for experiment {expnr}")
else:
    print(f"⚠ Directory {expdir} does not exist.")
    print("  Adjust the path to your simulation output directory.")
    # For demonstration, we'll continue without actual data
    sim = None

### Constructor Information

The `UDBase` constructor reads the following files:
- `namoptions.expnr`: Simulation input parameters
- `xxx.stl`: Urban geometry (optional)
- Preprocessing files: solid volumes, facet info, facet types, etc.

**Parameters:**
- `case_dir`: Path to the experiment directory
- Additional optional parameters for controlling what data to load

## 3. Accessing Simulation Properties

Once initialized, you can access simulation parameters and grid coordinates as properties.

In [ ]:
if sim is not None:
    # Domain size
    print("Domain size:")
    print(f"  xlen = {sim.xlen if hasattr(sim, 'xlen') else 'N/A'}")
    print(f"  ylen = {sim.ylen if hasattr(sim, 'ylen') else 'N/A'}")
    print(f"  zsize = {sim.zsize if hasattr(sim, 'zsize') else 'N/A'}")
    print()
    
    # Grid dimensions
    print("Grid dimensions:")
    print(f"  itot = {sim.itot}")
    print(f"  jtot = {sim.jtot}")
    print(f"  ktot = {sim.ktot}")
    print()
    
    # Grid spacing
    print("Grid spacing:")
    print(f"  dx = {sim.dx}")
    print(f"  dy = {sim.dy}")
    print()
    
    # Coordinates
    print("Grid coordinates available:")
    print(f"  sim.xm, sim.xt: x-coordinates (edges/centers)")
    print(f"  sim.ym, sim.yt: y-coordinates (edges/centers)")
    print(f"  sim.zm, sim.zt: z-coordinates (edges/centers)")
else:
    print("Example output (when data is available):")
    print("Domain size: 64m x 64m x 64m")
    print("Grid: 64 x 64 x 64")
    print("Grid spacing: dx=1.0, dy=1.0")

## 4. Geometry Visualization

If a geometry file (STL) is present, UDBase automatically loads it and provides access to geometry properties and visualization capabilities.

In [ ]:
if sim is not None and sim.geom is not None:
    print("Geometry properties:")
    print(f"  Number of faces: {sim.geom.n_faces}")
    print(f"  Number of vertices: {sim.geom.n_vertices}")
    print(f"  Total surface area: {sim.geom.total_area:.2f} m²")
    print(f"  Volume: {sim.geom.volume:.2f} m³")
    print()
    
    # Visualize geometry (uncomment to display)
    # sim.geom.show()
    # plt.xlabel('x [m]')
    # plt.ylabel('y [m]')
    # plt.show()
elif sim is not None:
    print("No geometry file loaded - flat terrain is assumed")
else:
    print("Geometry visualization example (when data is available)")
    print("  sim.geom.show()  # Opens 3D interactive viewer")

## 5. Available Methods

The UDBase class provides comprehensive methods for loading and analyzing simulation data:

**Field loading methods:**
- `load_field(var, time=None)` - Load 3D instantaneous field data
- `load_stat_xyt(var, time=None)` - Load 1D slab- and time-averaged statistics
- `load_stat_t(var)` - Load 3D time-averaged statistics
- `load_slice(slice_type, var, time=None)` - Load 2D slice data

**Facet loading methods:**
- `load_fac_momentum(time=None)` - Load momentum surface data
- `load_fac_eb(time=None)` - Load surface energy balance data
- `load_fac_temperature(time=None)` - Load facet temperature data
- `load_seb(time=None)` - Load all SEB terms

**Analysis methods:**
- `assign_prop_to_fac(prop)` - Assign properties to facets
- `area_average_fac(data, areas)` - Area-average over facets
- `area_average_seb(seb_dict, facet_type=None)` - Area-average SEB terms
- `time_average(data, axis=-1)` - Time-average data (static method)
- `convert_fac_to_field(fac_data, method='nearest')` - Convert facet to field
- `calculate_frontal_properties(direction='x')` - Calculate frontal area

**Visualization methods:**
- `plot_fac(data, title='', cmap='viridis')` - Plot facet data
- `plot_fac_type(title='Facet Types')` - Plot facet types

## 6. Quick Example Usage

Here's a typical workflow for analyzing simulation data:

In [ ]:
# Example workflow (requires actual simulation data)

# 1. Load field data
# u = sim.load_field('u', time=3600)
# print(f"Velocity field shape: {u.shape}")

# 2. Load facet data
# seb = sim.load_seb(time=3600)
# print(f"SEB components: {list(seb.keys())}")

# 3. Perform analysis
# avg_flux = sim.area_average_seb(seb)
# print(f"Average sensible heat flux: {avg_flux['qsens']:.2f} W/m²")

# frontal = sim.calculate_frontal_properties(direction='x')
# print(f"Frontal area ratio: {frontal['lambda_f']:.3f}")

# 4. Visualize
# sim.plot_fac(seb['qsens'], title='Sensible Heat Flux [W/m²]')
# plt.show()

print("Example workflow shown above (commented out)")
print("Uncomment and run when simulation data is available")

## 7. Summary and Next Steps

This tutorial introduced the **UDBase** class, which is the foundation for post-processing uDALES simulation data. You've learned how to:

- Initialize UDBase with a simulation directory
- Access simulation properties (domain size, grid dimensions, coordinates)
- Work with geometry data (if available)
- Understand the available methods for loading and analyzing data

### Next Steps

For detailed examples of working with specific data types, see:

- **fields_tutorial.ipynb** - Loading and analyzing field data (3D variables, slices, statistics)
- **facets_tutorial.ipynb** - Working with facet data (surface energy balance, momentum fluxes)
- **geometry_tutorial.ipynb** - Creating and manipulating urban geometries

### Additional Resources

- `tools/python/fields_example.py` - Complete field data examples
- `tools/python/facets_example.py` - Complete facet data examples
- `tools/python/QUICK_REFERENCE.py` - API quick reference guide
- uDALES documentation: https://u-dales.readthedocs.io/